In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, roc_auc_score
)
import joblib
import warnings
import os
import json
from datetime import datetime

warnings.filterwarnings('ignore')

# Set style for visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

class DepressionModelTrainer:
    def __init__(self, data_path, target_column='Depression', output_dir='models'):
        """
        Initialize the model trainer
        
        Args:
            data_path: Path to preprocessed data CSV
            target_column: Name of target variable
            output_dir: Directory to save models and results
        """
        self.data_path = data_path
        self.target_column = target_column
        self.output_dir = output_dir
        self.data = None
        self.X = None
        self.y = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.models = {}
        self.results = {}
        self.best_model = None
        self.best_model_name = None
        self.feature_names = None

        # Create output directories
        self._setup_directories()

    def _setup_directories(self):
        """Create necessary output directories"""
        dirs = [
            self.output_dir,
            f'{self.output_dir}/visualizations',
            f'{self.output_dir}/reports'
        ]
        for dir_path in dirs:
            os.makedirs(dir_path, exist_ok=True)
        print("✓ Output directories created")

    def load_preprocessed_data(self):
        """Load the preprocessed data"""
        print("\n" + "="*70)
        print("LOADING PREPROCESSED DATA")
        print("="*70)

        if not os.path.exists(self.data_path):
            print(f"✗ ERROR: File not found at {self.data_path}")
            print("   Please run the preprocessing script first.")
            return False

        try:
            self.data = pd.read_csv(self.data_path)
            print(f"✓ Data loaded successfully")
            print(f"  Shape: {self.data.shape}")
            print(f"  Columns: {len(self.data.columns)}")

            # Check if target column exists
            if self.target_column not in self.data.columns:
                print(f"\n✗ ERROR: Target column '{self.target_column}' not found!")
                print(f"  Available columns: {list(self.data.columns)[:5]}...")
                return False

            # Ensure target is integer
            self.data[self.target_column] = self.data[self.target_column].astype(int)

            # Verify binary classification
            unique_values = sorted(self.data[self.target_column].unique())
            if len(unique_values) != 2:
                print(f"\n✗ ERROR: Expected binary target, found {len(unique_values)} classes: {unique_values}")
                return False

            if set(unique_values) != {0, 1}:
                print(f"\n⚠️  WARNING: Target values are {unique_values}, expected [0, 1]")
                print("   Remapping to binary...")
                mapping = {unique_values[0]: 0, unique_values[1]: 1}
                self.data[self.target_column] = self.data[self.target_column].map(mapping)

            # Display distribution
            print(f"\n  Target Distribution ({self.target_column}):")
            value_counts = self.data[self.target_column].value_counts().sort_index()
            for val, count in value_counts.items():
                label = "Healthy (No Depression)" if val == 0 else "Depressed"
                percentage = (count / len(self.data)) * 100
                print(f"    {label} ({val}): {count:,} samples ({percentage:.1f}%)")

            # Check for class imbalance
            minority_class_pct = min(value_counts.values) / len(self.data) * 100
            if minority_class_pct < 20:
                print(f"\n  ⚠️  Class imbalance detected ({minority_class_pct:.1f}% minority class)")
                print("     Using stratified split and weighted metrics")

            return True

        except Exception as e:
            print(f"✗ ERROR loading data: {e}")
            import traceback
            traceback.print_exc()
            return False

    def prepare_data(self, test_size=0.2, random_state=42):
        """Prepare features and target, split into train/test"""
        print("\n" + "="*70)
        print("PREPARING DATA FOR TRAINING")
        print("="*70)

        try:
            # Separate features and target
            self.X = self.data.drop(columns=[self.target_column])
            self.y = self.data[self.target_column]
            self.feature_names = self.X.columns.tolist()

            print(f"  Features: {self.X.shape[1]} columns")
            print(f"  Samples: {len(self.X):,}")

            # Check for any remaining issues
            if self.X.isnull().any().any():
                print(f"  ⚠️  WARNING: Found {self.X.isnull().sum().sum()} missing values in features")
                print("     Filling with 0...")
                self.X = self.X.fillna(0)

            if np.isinf(self.X.values).any():
                print(f"  ⚠️  WARNING: Found infinite values in features")
                print("     Replacing with large finite values...")
                self.X = self.X.replace([np.inf, -np.inf], [1e10, -1e10])

            # Split data with stratification (maintains class balance)
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                self.X, self.y, 
                test_size=test_size, 
                random_state=random_state, 
                stratify=self.y
            )

            print(f"\n  Train/Test Split:")
            print(f"    Train set: {len(self.X_train):,} samples ({(1-test_size)*100:.0f}%)")
            print(f"    Test set:  {len(self.X_test):,} samples ({test_size*100:.0f}%)")

            # Verify stratification worked
            print(f"\n  Train distribution:")
            train_counts = self.y_train.value_counts().sort_index()
            for val, count in train_counts.items():
                pct = (count / len(self.y_train)) * 100
                print(f"    Class {val}: {count:,} ({pct:.1f}%)")

            print(f"\n  Test distribution:")
            test_counts = self.y_test.value_counts().sort_index()
            for val, count in test_counts.items():
                pct = (count / len(self.y_test)) * 100
                print(f"    Class {val}: {count:,} ({pct:.1f}%)")

            return True

        except Exception as e:
            print(f"✗ ERROR preparing data: {e}")
            import traceback
            traceback.print_exc()
            return False

    def initialize_models(self):
        """Initialize multiple ML models with optimized hyperparameters"""
        print("\n" + "="*70)
        print("INITIALIZING MODELS")
        print("="*70)

        self.models = {
            'Logistic Regression': LogisticRegression(
                max_iter=1000, 
                random_state=42,
                class_weight='balanced',
                solver='lbfgs'
            ),
            'Random Forest': RandomForestClassifier(
                n_estimators=100, 
                random_state=42, 
                max_depth=15,
                min_samples_split=10,
                min_samples_leaf=5,
                class_weight='balanced',
                n_jobs=-1
            ),
            'Gradient Boosting': GradientBoostingClassifier(
                n_estimators=100, 
                random_state=42, 
                max_depth=5,
                learning_rate=0.1,
                subsample=0.8
            ),
            'Support Vector Machine': SVC(
                probability=True, 
                random_state=42, 
                kernel='rbf',
                class_weight='balanced',
                C=1.0
            ),
            'K-Nearest Neighbors': KNeighborsClassifier(
                n_neighbors=7,
                weights='distance',
                metric='euclidean'
            ),
            'Decision Tree': DecisionTreeClassifier(
                random_state=42, 
                max_depth=15,
                min_samples_split=10,
                min_samples_leaf=5,
                class_weight='balanced'
            ),
            'Naive Bayes': GaussianNB()
        }

        print(f"  Initialized {len(self.models)} models:")
        for name in self.models.keys():
            print(f"    • {name}")

        return True

    def train_and_evaluate_models(self):
        """Train and evaluate all models with cross-validation"""
        print("\n" + "="*70)
        print("TRAINING AND EVALUATING MODELS")
        print("="*70 + "\n")

        for model_name, model in self.models.items():
            print(f"{'='*70}")
            print(f"MODEL: {model_name}")
            print(f"{'='*70}")

            try:
                # Train model
                print("Training...")
                model.fit(self.X_train, self.y_train)

                # Make predictions
                y_pred_test = model.predict(self.X_test)
                y_pred_train = model.predict(self.X_train)

                # Get probability predictions (for ROC curve)
                y_prob_test = None
                if hasattr(model, "predict_proba"):
                    y_prob_test = model.predict_proba(self.X_test)[:, 1]

                # Calculate metrics
                train_accuracy = accuracy_score(self.y_train, y_pred_train)
                test_accuracy = accuracy_score(self.y_test, y_pred_test)

                # Use weighted average for imbalanced datasets
                precision = precision_score(self.y_test, y_pred_test, average='weighted', zero_division=0)
                recall = recall_score(self.y_test, y_pred_test, average='weighted', zero_division=0)
                f1 = f1_score(self.y_test, y_pred_test, average='weighted', zero_division=0)

                # Calculate per-class metrics
                precision_per_class = precision_score(self.y_test, y_pred_test, average=None, zero_division=0)
                recall_per_class = recall_score(self.y_test, y_pred_test, average=None, zero_division=0)
                f1_per_class = f1_score(self.y_test, y_pred_test, average=None, zero_division=0)

                # ROC-AUC score
                roc_auc = None
                if y_prob_test is not None:
                    try:
                        roc_auc = roc_auc_score(self.y_test, y_prob_test)
                    except:
                        pass

                # Cross-validation score (CRITICAL FOR ROBUSTNESS)
                print("Performing 5-Fold Cross-Validation...")
                cv_scores = None
                cv_mean = None
                cv_std = None
                try:
                    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
                    cv_scores = cross_val_score(model, self.X_train, self.y_train, cv=cv, scoring='f1_weighted', n_jobs=-1)
                    cv_mean = cv_scores.mean()
                    cv_std = cv_scores.std()
                except Exception as e:
                    print(f"  ⚠️  Cross-validation failed: {str(e)[:50]}")

                # Confusion matrix
                cm = confusion_matrix(self.y_test, y_pred_test)

                # Store results
                self.results[model_name] = {
                    'model': model,
                    'train_accuracy': train_accuracy,
                    'test_accuracy': test_accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1,
                    'roc_auc': roc_auc,
                    'cv_scores': cv_scores,
                    'cv_mean': cv_mean,
                    'cv_std': cv_std,
                    'overfit_gap': train_accuracy - test_accuracy,
                    'y_pred_test': y_pred_test,
                    'y_prob_test': y_prob_test,
                    'precision_per_class': precision_per_class,
                    'recall_per_class': recall_per_class,
                    'f1_per_class': f1_per_class,
                    'confusion_matrix': cm
                }

                # PRINT DETAILED RESULTS
                print(f"\n📊 PERFORMANCE METRICS:")
                print(f"{'─'*70}")
                print(f"  Train Accuracy:  {train_accuracy*100:6.2f}%")
                print(f"  Test Accuracy:   {test_accuracy*100:6.2f}%")
                print(f"  Precision:       {precision*100:6.2f}%")
                print(f"  Recall:          {recall*100:6.2f}%")
                print(f"  F1-Score:        {f1*100:6.2f}%")
                if roc_auc:
                    print(f"  ROC-AUC:         {roc_auc*100:6.2f}%")
                if cv_mean is not None:
                    print(f"  CV F1-Score:     {cv_mean*100:6.2f}% (±{cv_std*100:4.2f}%)")
                    print(f"  CV Scores:       {[f'{s*100:.2f}%' for s in cv_scores]}")
                print(f"  Overfit Gap:     {(train_accuracy - test_accuracy)*100:6.2f}%")
                
                print(f"\n📈 PER-CLASS METRICS:")
                print(f"{'─'*70}")
                print(f"  Class 0 (Healthy):")
                print(f"    Precision: {precision_per_class[0]*100:6.2f}%")
                print(f"    Recall:    {recall_per_class[0]*100:6.2f}%")
                print(f"    F1-Score:  {f1_per_class[0]*100:6.2f}%")
                print(f"\n  Class 1 (Depressed):")
                print(f"    Precision: {precision_per_class[1]*100:6.2f}%")
                print(f"    Recall:    {recall_per_class[1]*100:6.2f}%")
                print(f"    F1-Score:  {f1_per_class[1]*100:6.2f}%")
                
                print(f"\n🎯 CONFUSION MATRIX:")
                print(f"{'─'*70}")
                print(f"                Predicted")
                print(f"              Healthy  Depressed")
                print(f"    Healthy      {cm[0][0]:4d}     {cm[0][1]:4d}")
                print(f"    Depressed    {cm[1][0]:4d}     {cm[1][1]:4d}")
                print()

                # Warning for overfitting
                if (train_accuracy - test_accuracy) > 0.1:
                    print(f"⚠️  WARNING: Possible overfitting detected!")
                    print(f"   Train-Test gap: {(train_accuracy - test_accuracy)*100:.2f}%\n")

            except Exception as e:
                print(f"✗ ERROR training {model_name}: {e}")
                import traceback
                traceback.print_exc()
                continue

        if not self.results:
            print("\n✗ ERROR: No models trained successfully!")
            return False

        print(f"\n✓ Successfully trained {len(self.results)}/{len(self.models)} models")
        return True

    def compare_models(self):
        """Compare all models and find the best one"""
        print("\n" + "="*70)
        print("FINAL MODEL COMPARISON")
        print("="*70 + "\n")

        # Create comparison dataframe
        comparison_data = []
        for model_name, results in self.results.items():
            row = {
                'Model': model_name,
                'Test Accuracy': results['test_accuracy'],
                'F1-Score': results['f1_score'],
                'Precision': results['precision'],
                'Recall': results['recall'],
                'ROC-AUC': results['roc_auc'] if results['roc_auc'] else 0,
                'CV F1-Score': results['cv_mean'] if results['cv_mean'] else 0,
                'CV Std': results['cv_std'] if results['cv_std'] else 0,
                'Overfit Gap': results['overfit_gap']
            }
            comparison_data.append(row)

        comparison_df = pd.DataFrame(comparison_data)

        # Sort by F1-Score (better for medical data than accuracy)
        comparison_df = comparison_df.sort_values('F1-Score', ascending=False)

        # Print formatted table
        print(f"{'Model':<25} {'Accuracy':>10} {'F1-Score':>10} {'Precision':>10} {'Recall':>10} {'CV F1':>10}")
        print("─" * 95)
        for _, row in comparison_df.iterrows():
            print(f"{row['Model']:<25} {row['Test Accuracy']*100:>9.2f}% {row['F1-Score']*100:>9.2f}% "
                  f"{row['Precision']*100:>9.2f}% {row['Recall']*100:>9.2f}% {row['CV F1-Score']*100:>9.2f}%")

        # Find best model
        self.best_model_name = comparison_df.iloc[0]['Model']
        self.best_model = self.results[self.best_model_name]['model']

        print(f"\n{'='*70}")
        print(f"🏆 BEST MODEL: {self.best_model_name}")
        print(f"{'='*70}")
        best_res = self.results[self.best_model_name]
        print(f"  Test Accuracy:   {best_res['test_accuracy']*100:.2f}%")
        print(f"  F1-Score:        {best_res['f1_score']*100:.2f}%")
        print(f"  Precision:       {best_res['precision']*100:.2f}%")
        print(f"  Recall:          {best_res['recall']*100:.2f}%")
        if best_res['roc_auc']:
            print(f"  ROC-AUC:         {best_res['roc_auc']*100:.2f}%")
        if best_res['cv_mean']:
            print(f"  CV F1-Score:     {best_res['cv_mean']*100:.2f}% (±{best_res['cv_std']*100:.2f}%)")
        print()

        # Save comparison to CSV
        comparison_df.to_csv(f'{self.output_dir}/model_comparison.csv', index=False)
        print(f"✓ Comparison saved to: {self.output_dir}/model_comparison.csv\n")

        return comparison_df

    def generate_detailed_report(self):
        """Generate detailed classification report for best model"""
        print("\n" + "="*70)
        print("GENERATING DETAILED REPORT")
        print("="*70)

        best_results = self.results[self.best_model_name]
        
        # Classification report
        report = classification_report(
            self.y_test, 
            best_results['y_pred_test'],
            target_names=['Healthy', 'Depressed'],
            digits=4
        )

        print(f"\nClassification Report for {self.best_model_name}:")
        print("─" * 70)
        print(report)

        # Save to file
        report_path = f'{self.output_dir}/reports/classification_report.txt'
        with open(report_path, 'w') as f:
            f.write(f"Classification Report - {self.best_model_name}\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            f.write(report)
            f.write(f"\n\nConfusion Matrix:\n")
            f.write(str(best_results['confusion_matrix']))
            if best_results['cv_mean']:
                f.write(f"\n\nCross-Validation F1-Scores:\n")
                f.write(f"Mean: {best_results['cv_mean']:.4f}\n")
                f.write(f"Std:  {best_results['cv_std']:.4f}\n")
                f.write(f"Individual Scores: {best_results['cv_scores']}\n")

        print(f"✓ Report saved to: {report_path}")

        # Save comprehensive metrics as JSON
        metrics = {
            'model_name': self.best_model_name,
            'timestamp': datetime.now().isoformat(),
            'test_accuracy': float(best_results['test_accuracy']),
            'train_accuracy': float(best_results['train_accuracy']),
            'f1_score': float(best_results['f1_score']),
            'precision': float(best_results['precision']),
            'recall': float(best_results['recall']),
            'roc_auc': float(best_results['roc_auc']) if best_results['roc_auc'] else None,
            'cv_f1_mean': float(best_results['cv_mean']) if best_results['cv_mean'] else None,
            'cv_f1_std': float(best_results['cv_std']) if best_results['cv_std'] else None,
            'cv_f1_scores': [float(s) for s in best_results['cv_scores']] if best_results['cv_scores'] is not None else None,
            'overfit_gap': float(best_results['overfit_gap']),
            'confusion_matrix': best_results['confusion_matrix'].tolist(),
            'per_class_metrics': {
                'healthy': {
                    'precision': float(best_results['precision_per_class'][0]),
                    'recall': float(best_results['recall_per_class'][0]),
                    'f1': float(best_results['f1_per_class'][0])
                },
                'depressed': {
                    'precision': float(best_results['precision_per_class'][1]),
                    'recall': float(best_results['recall_per_class'][1]),
                    'f1': float(best_results['f1_per_class'][1])
                }
            }
        }

        metrics_path = f'{self.output_dir}/best_model_metrics.json'
        with open(metrics_path, 'w') as f:
            json.dump(metrics, f, indent=2)

        print(f"✓ Metrics saved to: {metrics_path}\n")

    def plot_visualizations(self):
        """Generate comprehensive visualizations"""
        print("\n" + "="*70)
        print("GENERATING VISUALIZATIONS")
        print("="*70)

        viz_dir = f'{self.output_dir}/visualizations'

        # 1. Confusion Matrix for Best Model
        try:
            best_results = self.results[self.best_model_name]
            cm = best_results['confusion_matrix']

            plt.figure(figsize=(8, 6))
            sns.heatmap(
                cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Healthy', 'Depressed'],
                yticklabels=['Healthy', 'Depressed'],
                cbar_kws={'label': 'Count'}
            )
            plt.title(f'Confusion Matrix: {self.best_model_name}', fontsize=14, fontweight='bold')
            plt.ylabel('True Label', fontsize=12)
            plt.xlabel('Predicted Label', fontsize=12)
            
            # Add percentages
            total = cm.sum()
            for i in range(2):
                for j in range(2):
                    pct = (cm[i, j] / total) * 100
                    plt.text(j + 0.5, i + 0.7, f'({pct:.1f}%)', 
                            ha='center', va='center', fontsize=10, color='gray')
            
            plt.tight_layout()
            plt.savefig(f'{viz_dir}/confusion_matrix_best.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("  ✓ Saved confusion matrix")

        except Exception as e:
            print(f"  ⚠️  Error plotting confusion matrix: {e}")

        # 2. ROC Curves for all models
        try:
            plt.figure(figsize=(10, 8))
            
            for name, res in self.results.items():
                if res['y_prob_test'] is not None:
                    fpr, tpr, _ = roc_curve(self.y_test, res['y_prob_test'])
                    auc = res['roc_auc']
                    
                    linestyle = '-' if name == self.best_model_name else '--'
                    linewidth = 3 if name == self.best_model_name else 2
                    
                    plt.plot(fpr, tpr, linestyle=linestyle, linewidth=linewidth,
                            label=f'{name} (AUC = {auc:.3f})')

            plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random (AUC = 0.500)')
            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('ROC Curves - All Models', fontsize=14, fontweight='bold')
            plt.legend(loc='lower right', fontsize=10)
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.savefig(f'{viz_dir}/roc_curves.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("  ✓ Saved ROC curves")

        except Exception as e:
            print(f"  ⚠️  Error plotting ROC curves: {e}")

        # 3. Model Comparison Bar Chart
        try:
            metrics_data = []
            for name, res in self.results.items():
                metrics_data.append({
                    'Model': name,
                    'Accuracy': res['test_accuracy'],
                    'Precision': res['precision'],
                    'Recall': res['recall'],
                    'F1-Score': res['f1_score']
                })

            df_metrics = pd.DataFrame(metrics_data)
            df_melted = df_metrics.melt(id_vars='Model', var_name='Metric', value_name='Score')

            plt.figure(figsize=(14, 6))
            sns.barplot(data=df_melted, x='Model', y='Score', hue='Metric')
            plt.title('Model Performance Comparison', fontsize=14, fontweight='bold')
            plt.xlabel('Model', fontsize=12)
            plt.ylabel('Score', fontsize=12)
            plt.ylim(0, 1.05)
            plt.xticks(rotation=45, ha='right')
            plt.legend(title='Metric', loc='lower right')
            plt.grid(axis='y', alpha=0.3)
            plt.tight_layout()
            plt.savefig(f'{viz_dir}/model_comparison.png', dpi=300, bbox_inches='tight')
            plt.close()
            print("  ✓ Saved model comparison chart")

        except Exception as e:
            print(f"  ⚠️  Error plotting model comparison: {e}")

        # 4. Feature Importance (if available)
        try:
            if hasattr(self.best_model, 'feature_importances_'):
                importances = self.best_model.feature_importances_
                indices = np.argsort(importances)[-20:]  # Top 20

                plt.figure(figsize=(10, 8))
                plt.barh(range(len(indices)), importances[indices])
                plt.yticks(range(len(indices)), [self.feature_names[i] for i in indices])
                plt.xlabel('Importance', fontsize=12)
                plt.title(f'Top 20 Feature Importances: {self.best_model_name}', 
                         fontsize=14, fontweight='bold')
                plt.tight_layout()
                plt.savefig(f'{viz_dir}/feature_importance.png', dpi=300, bbox_inches='tight')
                plt.close()
                print("  ✓ Saved feature importance chart")

        except Exception as e:
            print(f"  ⚠️  Feature importance not available")

        print()

    def save_best_model(self):
        """Save the best model and ALL related artifacts (CRITICAL FOR APP)"""
        print("\n" + "="*70)
        print("SAVING BEST MODEL AND ARTIFACTS")
        print("="*70)

        # 1. Save model with generic name (for easy loading in apps)
        generic_path = f'{self.output_dir}/best_model.pkl'
        joblib.dump(self.best_model, generic_path)
        print(f"  ✓ Best model saved: {generic_path}")

        # 2. Save model with specific name (for backup/version control)
        specific_name = self.best_model_name.replace(" ", "_").lower()
        specific_path = f'{self.output_dir}/model_{specific_name}.pkl'
        joblib.dump(self.best_model, specific_path)
        print(f"  ✓ Backup saved: {specific_path}")

        # 3. Save feature names (CRITICAL FOR APP INPUTS)
        if self.feature_names:
            features_path = f'{self.output_dir}/feature_names.json'
            with open(features_path, 'w') as f:
                json.dump(self.feature_names, f, indent=2)
            print(f"  ✓ Feature names saved: {features_path}")

        # 4. Save model info text
        info_path = f'{self.output_dir}/best_model_info.txt'
        with open(info_path, 'w') as f:
            res = self.results[self.best_model_name]
            f.write(f"Best Model: {self.best_model_name}\n")
            f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            f.write(f"Test Accuracy:  {res['test_accuracy']:.4f}\n")
            f.write(f"F1-Score:       {res['f1_score']:.4f}\n")
            if res['cv_mean']:
                f.write(f"CV F1-Score:    {res['cv_mean']:.4f} (±{res['cv_std']:.4f})\n")

        print(f"  ✓ Info saved: {info_path}\n")

    def run_pipeline(self, specific_name=None):
        """Run the complete training pipeline"""
        print("\n" + "🚀 STARTING MODEL TRAINING PIPELINE 🚀".center(70))
        
        start_time = datetime.now()

        if not self.load_preprocessed_data(): return False
        if not self.prepare_data(): return False
        if not self.initialize_models(): return False
        if not self.train_and_evaluate_models(): return False

        self.compare_models()
        self.generate_detailed_report()
        self.plot_visualizations()
        self.save_best_model()

        duration = (datetime.now() - start_time).total_seconds()

        print("="*70)
        print("✅ TRAINING COMPLETED SUCCESSFULLY! ✅".center(70))
        print("="*70)
        print(f"\nExecution time: {duration:.1f} seconds")
        print(f"\nOutput files created in {self.output_dir}/:")
        print(f"  📁 best_model.pkl")
        print(f"  📁 model_{specific_name}.pkl")
        print(f"  📁 feature_names.json")
        print(f"  📁 best_model_info.txt")
        print(f"  📁 best_model_metrics.json")
        print(f"  📁 model_comparison.csv")
        print(f"  📁 visualizations/")
        print(f"  📁 reports/")
        print()
        return True


# MAIN EXECUTION
if __name__ == "__main__":
    # Ensure this matches your file path
    PREPROCESSED_DATA = "data/processed/preprocessed_data.csv"
    
    trainer = DepressionModelTrainer(PREPROCESSED_DATA)
    trainer.run_pipeline()

✓ Output directories created

                 🚀 STARTING MODEL TRAINING PIPELINE 🚀                 

LOADING PREPROCESSED DATA
✓ Data loaded successfully
  Shape: (27901, 63)
  Columns: 63

  Target Distribution (Depression):
    Healthy (No Depression) (0): 11,565 samples (41.5%)
    Depressed (1): 16,336 samples (58.5%)

PREPARING DATA FOR TRAINING
  Features: 62 columns
  Samples: 27,901

  Train/Test Split:
    Train set: 22,320 samples (80%)
    Test set:  5,581 samples (20%)

  Train distribution:
    Class 0: 9,252 (41.5%)
    Class 1: 13,068 (58.5%)

  Test distribution:
    Class 0: 2,313 (41.4%)
    Class 1: 3,268 (58.6%)

INITIALIZING MODELS
  Initialized 7 models:
    • Logistic Regression
    • Random Forest
    • Gradient Boosting
    • Support Vector Machine
    • K-Nearest Neighbors
    • Decision Tree
    • Naive Bayes

TRAINING AND EVALUATING MODELS

MODEL: Logistic Regression
Training...
Performing 5-Fold Cross-Validation...

📊 PERFORMANCE METRICS:
─────────────────